<h1>Simple examples of how Monte Carlo simulation can be used</h1>
<p>Statisitics problems are often challenging in cases where multiple interacting probabalistic entities interact.  In statistics, we often start with either rolling dice.  We'll start by creating our dice simulators.</p>

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import random
import numpy as np
from math import sqrt

def roll_fair(n_face=6):
    return random.randint(1,n_face)
    
def two_dice_pmf():
    res = [0.]*11
    for i in range(1,7):
        for j in range(1,7):
            res[i+j-2] += 1.
    return map(lambda x: x/np.sum(res), res)

def roll_two():
    return roll_fair() + roll_fair()

<h2>Let's start with a simple multinomial probability</h2>

In [ ]:
data = [
    go.Bar(
        name="Multinomial PMF",
        x=range(2,13),
        y=two_dice_pmf(),
        marker=dict(
            color='#990000'
        )
    )
    
]


layout = go.Layout(
    title='2 Dice PMF',
    xaxis=dict(
        title='Roll Result',
        titlefont=dict(
            size=14,
            color='black'
        )
    ),
    yaxis=dict(
        title='Probability',
        titlefont=dict(
            size=14,
            color='black'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='dice_pmf')

In [ ]:
def roll_results(n_rolls):
    res = []
    for i in range(n_rolls):
        res.append(roll_two())
    
    occ, edges = np.histogram(res, bins=map(lambda x: x+0.5, range(1,13)), density=True)
    err = map(lambda x: sqrt(((1-x)*x)/n_rolls), occ)
    return (occ, edges, err)

occ2, edges, err2 = roll_results(100)
occ4, edges, err4 = roll_results(10000)
bins = map(lambda x: x+.5, edges)    
    
data = [
    go.Scatter(
        name="100 Trials",
        x=bins,
        y=occ2,
        mode = 'markers',
        marker=dict(
            color='black',
        ),
        error_y=dict(
            type='data',
            array=err2,
            visible=True,
            color='black'
        )
    ),
    go.Scatter(
        name="10,000 Trials",
        x=bins,
        y=occ4,
        mode = 'markers',
        marker=dict(
            color='green',
        ),
        error_y=dict(
            type='data',
            array=err4,
            visible=True,
            color='green'
        )
    ),
    go.Bar(
        name="Multinomial PMF",
        x=bins,
        y=two_dice_pmf(),
        marker=dict(
            color='#990000'
        )
    )
    
]

layout = go.Layout(
    title='2 Dice PMF w/ Simulated trials',
    xaxis=dict(
        title='Roll Result',
        titlefont=dict(
            size=14,
            color='black'
        )
    ),
    yaxis=dict(
        title='Probability',
        titlefont=dict(
            size=14,
            color='black'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='dice_pmf_w_sim')

In [ ]:
def interacting_dice_pmf():
    res = [0.]*11
    for i in range(1,7):
        for j in range(1,7):
            if i+j < 7:
                for k in range(1,7):
                    res[k+j-2] += 1.
            else:
                #because the other branch has 6 subtrees, 
                #every possibility must have equal weight
                res[i+j-2] += 6.
    return map(lambda x: x/np.sum(res), res)

def int_roll_results(n_rolls):
    res = []
    for i in range(n_rolls):
        r1 = roll_fair()
        r2 = roll_fair()
        if r1+r2 < 7:
            r1 = roll_fair()
        res.append(r1+r2)
    
    occ, edges = np.histogram(res, bins=map(lambda x: x+0.5, range(1,13)), density=True)
    err = map(lambda x: sqrt(((1-x)*x)/n_rolls), occ)
    return (occ, edges, err)

In [ ]:
def roll_results(n_rolls):
    res = []
    for i in range(n_rolls):
        res.append(roll_two())
    
    occ, edges = np.histogram(res, bins=map(lambda x: x+0.5, range(1,13)), density=True)
    err = map(lambda x: ((1-x)*x)/n_rolls, occ)
    return (occ, edges, err)

occ2, edges, err2 = int_roll_results(1000)
occ4, edges, err4 = int_roll_results(10000)
bins = map(lambda x: x+.5, edges)    
    
data = [
    go.Scatter(
        name="1000 Trials",
        x=bins,
        y=occ2,
        mode = 'markers',
        marker=dict(
            color='black',
        ),
        error_y=dict(
            type='data',
            array=err2,
            visible=True,
            color='black'
        )
    ),
    go.Scatter(
        name="10,000 Trials",
        x=bins,
        y=occ4,
        mode = 'markers',
        marker=dict(
            color='green',
        ),
        error_y=dict(
            type='data',
            array=err4,
            visible=True,
            color='green'
        )
    ),
    go.Bar(
        name="Multinomial PMF",
        x=bins,
        y=interacting_dice_pmf(),
        marker=dict(
            color='#990000'
        )
    )
    
]

layout = go.Layout(
    title='2 Interacting Dice PMF w/ Simulated trials',
    xaxis=dict(
        title='Roll Result',
        titlefont=dict(
            size=14,
            color='black'
        )
    ),
    yaxis=dict(
        title='Probability',
        titlefont=dict(
            size=14,
            color='black'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='int_dice_pmf_w_sim')